In [2]:
from requests_html import HTML
import requests
from bs4 import BeautifulSoup
import csv
# URL bağlantısı çekmek istediğim linke özel olmadığı için html kodlarını indirdim.
# str formatında olan bu dizinler html kodu analiz eder gibi davranabilirim.
with open("duyogr.html") as html_file:
 
    # sayfayı indirmek değil, önemli olan kaynak kodunu yazılı formatta almak
    # kaynak kodunu kopyala yapıştır yazınız
    # bahsi geçen html dosyası sonuçları aldıktan sonra url kaynağından alınmış kodları içermektedir.

    source = html_file.read()
    html1 = HTML(html=source)

# burda verdiğim değişken adları o ana verilmiş değişkenlerdir.
# özel bir anlam ifade etmemektedir
    match = html1.find("div.shell", first=True)

    kaynakKodu = match.find("script", first=True).html
"""
HTML kodu üzerinde istediğim alan javascript 
formatında kayıtlı olduğu için böyle bir yol izledim
"""
# her tez detayının önenünde 'tezDetay' yazdığını kodlar arasından gördüm.
# uzunluklarının aynı olması işimi inanılmaz kolaylaştırdı.
# Yök Tez sitesinde her kodun detay kısmında ki 'ID' ve 'NO' aynı uzunluktadır.
# print(bir.split("tezDetay")[1][2:24])
# print(bir.split("tezDetay")[1][27:49])

i = 1
for i in range(1, 63):
    link_id = kaynakKodu.split("tezDetay")[i][2:24]
    link_no = kaynakKodu.split("tezDetay")[i][27:49]
# Range i'yi kaynak koddaki maximum sayım kadar kullanınız.
# döngüde benim sonucumda 172 tane sonuc olduğu için max range 173 kullanıldı.

#Her tez özeti için özel url vardır.
# örnek link
# https://tez.yok.gov.tr/UlusalTezMerkezi/tezDetay.jsp?id="TBzuU3MqYhqJ9XHc-esfgA"&no="TBzuU3MqYhqJ9XHc-esfgA"

    url = 'https://tez.yok.gov.tr/UlusalTezMerkezi/tezDetay.jsp?id="' +link_id+'"&no="'+link_no
# URL işlemini otomatize ettikten sonra bu aşamada oluşturduğumuz url'lerden html kodlarını alma işlemi kalıyor.

    R = requests.get(url)
    doc = BeautifulSoup(R.text, "html.parser")
# HTML kodlarını düzenli bir biçimde görmek için 'prettify' komutunu kullanıyoruz.
# print(doc.prettify())
# yök tez atlas sonuc sayfasından aldığınız tüm tez sonuçlarını için bu adımları uygulabilirsiniz

    tag = doc.find_all("td")
    # print(tag[])
    tezNo = tag[4].text.strip()
    print(tezNo)
    tezKunye = str(tag[6])
    tezDurumu = str(tag[7])
    tezOzetTR = (tag[9].text.strip())
    tezOzetING = (tag[11].text.strip())

    tezAdıTR = (tezKunye.split("<br/>")[0].split("/")[1].strip())

    tezAdıING = (tezKunye.split("<br/>")
                 [0].split("/")[0].split('<td valign="top">')[1].strip())
#!!ÖNEMLİ NOT:
# tezAdıING, Türkçe tez adları içermektedir.Bu sisteme yüklenme şekliyle ilgilidir.
# tezAdıING ve tezAdıTR değişkenleri çoğunluğa göre seçilmişitir. Düzeltmeyi yapma yolunu bulmadım.
# seçilen dillerin haricinde diller içerebilir.
    tezYazarAdı = (tezKunye.split("<br/>")[1].split(":")[1])
    tezDanısmanAdı = (tezKunye.split("<br/>")[2].split(":")[1].strip())+";"
    tezDanismanAdi1 = (tezDanısmanAdı.split(";")[0])
    tezDanismanAdi2 = (tezDanısmanAdı.split(";")[1])
    tezYerBilgisi = (tezKunye.split("<br/>")[3].split(":")[1].strip())
    tezLisansDerecesi = (tezDurumu.split("<br/>")[1].strip())
    tezYayimTarihi = (tezDurumu.split("<br/>")[2].strip())
    tezTarihi = (tezDurumu.split("<br/>")[3].strip())
    a = 0
    tezKonusuTR=[]
    tezKonusuING=[]
    for a in range(0, len(tezKunye.split("<br/>")[4].split(";"))):
        tezKonusuTR.reverse()
        tezKonusuTR=[tezKonusuTR]
        
        tezKonusuTR.append(tezKunye.split("<br/>")
                        [4].split(";")[a].split("=")[0].strip())
        tezKonusuING=[tezKonusuING]
        tezKonusuING.append(tezKunye.split("<br/>")
                        [4].split(";")[a].split("=")[1].strip())
        
        a += 1
    with open("duygusal.csv", "a", newline="") as csvFile:
        fieldNames = ["No", "Adı_TR", "Adı_ING", "Yazar_Adı", "1.Danisman_Adı", "2.Danisman_Adı",
                      "Yer_Bilgisi", "Lisans_Derecesi", "Yayim_Tarihi",
                      "Konusu_TR", "Konusu_ING", "Ozeti_TR", "Ozeti_ING", "Tez_tarihi"]
        thewriter = csv.DictWriter(csvFile, fieldnames=fieldNames)
        thewriter.writeheader()
        thewriter.writerow({"No": [tezNo], "Adı_TR": [tezAdıTR],
                            "Adı_ING": [tezAdıING], "Yazar_Adı": [tezYazarAdı],
                            "1.Danisman_Adı": [tezDanismanAdi1], "2.Danisman_Adı": [tezDanismanAdi2],
                            "Yer_Bilgisi": [tezYerBilgisi], "Lisans_Derecesi": [tezLisansDerecesi],
                            "Yayim_Tarihi": [tezYayimTarihi],
                            "Konusu_TR": [tezKonusuTR], "Konusu_ING": [tezKonusuING],
                            "Ozeti_TR": [tezOzetTR], "Ozeti_ING": [tezOzetING], "Tez_tarihi": [tezTarihi]}) 
    # print(tezOzetING)
    tezKonusuTR=[]
    tezKonusuING=[]   
    print(tezKonusuTR)
    print(tezKonusuING)
    i+=1


698377
[]
[]
756594
[]
[]
694775
[]
[]
683879
[]
[]
580714
[]
[]
506675
[]
[]
321177
[]
[]
745973
[]
[]
734240
[]
[]
734179
[]
[]
733950
[]
[]
743944
[]
[]
747576
[]
[]
750047
[]
[]
750490
[]
[]
760189
[]
[]
748202
[]
[]
711602
[]
[]
720700
[]
[]
739123
[]
[]
711808
[]
[]
722084
[]
[]
674060
[]
[]
697209
[]
[]
702206
[]
[]
700848
[]
[]
698925
[]
[]
674196
[]
[]
681067
[]
[]
693491
[]
[]
687787
[]
[]
701600
[]
[]
693490
[]
[]
692446
[]
[]
670117
[]
[]
694138
[]
[]
702448
[]
[]
695548
[]
[]
700573
[]
[]
673025
[]
[]
678912
[]
[]
683856
[]
[]
693446
[]
[]
685229
[]
[]
678305
[]
[]
678413
[]
[]
690030
[]
[]
663940
[]
[]
700554
[]
[]
694953
[]
[]
618631
[]
[]
698761
[]
[]
659504
[]
[]


Error: failed to execute this cell, please try again.

In [5]:
!pip install requests_html
!pip install request
!pip install bs4

  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached parse-1.19.0-py3-none-any.whl
  Using cached fake_useragent-1.1.1-py3-none-any.whl (50 kB)
  Using cached w3lib-2.1.1-py3-none-any.whl (21 kB)
  Using cached pyppeteer-1.0.2-py3-none-any.whl (83 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached bs4-0.0.1-py3-none-any.whl
  Using cached pyquery-1.4.3-py3-none-any.whl (22 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Using cached websockets-10.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (106 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pyee-8.2.2-py2.py3-none-any.whl (12 kB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached impo